In [1]:
#%run input/Format.ipynb
import ROOT as root
from array import array
root.gErrorIgnoreLevel = root.kFatal
import pandas as pd
%jsroot on

Welcome to JupyROOT 6.30/06


In [2]:
iOption0 = 0 # piminus, electron

In [3]:
colors=[1,2,4,root.kGreen+2,root.kMagenta,root.kOrange+4,root.kGray,root.kCyan]
central_bins = [0,95,0,20,20,40,40,60,60,100]
N_centr = int(len(central_bins)/2)
pt_boarders = [0.2,0.3,0.4,0.5,0.6,0.7,0.9,1.1,1.4,1.9,2.5,3.5,5.0]
N_pt = len(pt_boarders)-1
file_path="/home/yoren/bnl/PHENIX/ee/AuAu/"
file_names=["my-100M_piminus_single_hagedorn_04-5_v0.root","my-10M_electron_embed_helios_v3.root","my-10M_bbbar_pythia_v7.root"]
code_path="../AnaTrain/Run14AuAuLeptonComby/"
file_name=file_names[iOption0]
outfilenames= ['pionminus','electron','positron']#output
filename=outfilenames[iOption0]
part_ids=[9,3,2]
part_id = part_ids[iOption0]
part_name = ["e^{+}","e^{-}"]

In [4]:
root.gSystem.Load(code_path+"MyEvent_C.so")

0

In [5]:
root.gROOT.ProcessLine(
f"TFile *input = new TFile(\"{file_path+file_name}\", \"READ\");\
TTree *T = (TTree *)input->Get(\"tree\");\
MyDileptonAnalysis::MyEvent *myevent = 0;\
TBranch *br = T->GetBranch(\"MyEvent\");\
br->SetAddress(&myevent);\
MyDileptonAnalysis::MyEventContainer *event_container = new MyDileptonAnalysis::MyEventContainer();\
event_container->InitEvent();\
event_container->GetHistsFromFile(\"{code_path}field_map.root\");\
"
); 
from ROOT import myevent, br, event_container

File opened at ../AnaTrain/Run14AuAuLeptonComby/field_map.root


mcid: 2 - positron, 3 - electron, 8 - pion+, 9 - pion minus

In [6]:
geant3dict = {
  2: "positron",
  3: "electron",
  5: "+muon",
  6: "-muon",
  8: "+pion",
  9: "-pion",
  11: "+kaon",
  12: "-kaon",
  14: "proton",
  15: "antiproton"
}

In [7]:
N_real_ev = br.GetEntries()
print(N_real_ev)
good_event_counter = 0
soso_event_counter = 0 
for ievent in range(N_real_ev):
    if good_event_counter>100: break
    myevent.ClearEvent()
    br.GetEntry(ievent)
    event_container.SetEvent(myevent)
    if not myevent.GetNtrack(): continue

    counter = 1
    while counter< myevent.GetNVTXhit()-1:
        for ihit in range(myevent.GetNVTXhit()):
            if myevent.GetVTXHitEntry(ihit).GetSensor() == 1 :
                #print("kek")
                myevent.RemoveVTXHitEntry(ihit)
                break
            else :
                counter=ihit
            
    n_hits = myevent.GetNVTXhit()
    for ihit in range(n_hits):
        if myevent.GetVTXHitEntry(ihit).GetSensor() == 1 :
            myevent.RemoveVTXHitEntry(ihit)

    n_hits = myevent.GetNVTXhit()
    for ihit in range(n_hits):
        if myevent.GetVTXHitEntry(ihit).GetSensor() == 1 :
            print("keks")
            myevent.RemoveVTXHitEntry(ihit)
    #for ihit in range(n_hits):
    #    hit1 = myevent.GetVTXHitEntry(ihit)
    #    for jhit in range(ihit+1,n_hits):
    #        hit2 = myevent.GetVTXHitEntry(jhit)
    #        if  abs(hit1.GetXHit() - hit2.GetXHit())<0.0001  and abs(hit1.GetYHit() - hit2.GetYHit())<0.0001 and abs(hit1.GetZHit() - hit2.GetZHit())<0.0001 :
    #            print(ihit,jhit,hit2.GetSensor(),hit1.GetSensor())
    #            if hit2.GetSensor()>0 :
    #                myevent.RemoveVTXHitEntry(jhit)
    #                n_hits-=1
    #                jhit-=1
    #                continue
                #else: 
                #    if hit1.GetSensor()>0 :
                #        myevent.RemoveVTXHitEntry(ihit)
                #        n_hits-=1
                #        ihit-=1
                #        continue
    
    event_container.correct_beam_offset()
    event_container.Associate_Hits_to_Leptons(5,5,5)
    soso_event_counter+=1
    do_continue = False
    if event_container.GetNBDThit()>0:
        do_continue = True
    if not do_continue:
        continue
    good_event_counter+=1
    print(f"Nevent: {ievent}, centrality: {myevent.GetCentrality()}, N VTX hit: {myevent.GetNVTXhit()}, Ntracks: {myevent.GetNtrack()}, NBDTtracks: {event_container.GetNBDThit()}") 

    for ipart in range(event_container.GetNBDThit()):
        print("N hits:",  event_container.GetBDTHitEntry(ipart).GetNBDThit(), ", pt:", round(event_container.GetBDTHitEntry(ipart).GetPt(),3),\
              ", id:",  geant3dict[myevent.GetEntry(ipart).GetMcId()])
        for ibdthit in range(event_container.GetBDTHitEntry(ipart).GetNBDThit()):
            bdthit = event_container.GetBDTHitEntry(ipart).GetBDTHitEntry(ibdthit)
            print(ibdthit, round(bdthit.Getsdphi(0),3),",",round(bdthit.Getsdphi(1),3),",",round(bdthit.Getsdphi(2),3),",",round(bdthit.Getsdphi(3),3),";",\
                  round(bdthit.GetIsTrue(0),3),",",round(bdthit.GetIsTrue(1),3),",",round(bdthit.GetIsTrue(2),3),",",round(bdthit.GetIsTrue(3),3))
    for ipart in range(myevent.GetNgentrack()):
        print(myevent.GetGenTrack(ipart).GetPt())
print(good_event_counter,soso_event_counter)

99748438
Nevent: 14, centrality: 64.0, N VTX hit: 3, Ntracks: 1, NBDTtracks: 1
N hits: 1 , pt: 1.276 , id: -pion
0 -0.18 , -0.235 , 999.0 , 1.634 ; 1.0 , 1.0 , 1.0 , 0.0
1.2659282684326172
Nevent: 20, centrality: 20.0, N VTX hit: 13, Ntracks: 1, NBDTtracks: 1
N hits: 3 , pt: 0.843 , id: -pion
0 -0.629 , -0.295 , 1.471 , 999.0 ; 1.0 , 1.0 , 1.0 , 0.0
1 -0.629 , -0.295 , 0.808 , 1.339 ; 1.0 , 1.0 , 1.0 , 1.0
2 -0.629 , 0.288 , 999.0 , 1.339 ; 1.0 , 1.0 , 1.0 , 0.0
0.8354507088661194
Nevent: 27, centrality: 1.0, N VTX hit: 37, Ntracks: 1, NBDTtracks: 1
N hits: 2 , pt: 0.657 , id: -pion
0 0.858 , 0.229 , 999.0 , -0.045 ; 1.0 , 1.0 , 1.0 , 0.0
1 -2.871 , 2.738 , 999.0 , -0.045 ; 1.0 , 0.0 , 1.0 , 0.0
0.6536231637001038
Nevent: 28, centrality: 30.0, N VTX hit: 20, Ntracks: 1, NBDTtracks: 1
N hits: 6 , pt: 0.673 , id: -pion
0 0.109 , 1.412 , -1.305 , 999.0 ; 1.0 , 1.0 , 1.0 , 0.0
1 1.891 , -0.234 , -1.305 , 999.0 ; 1.0 , 0.0 , 1.0 , 0.0
2 0.109 , 1.412 , -0.121 , -1.247 ; 1.0 , 1.0 , 1.0 , 1.

In [8]:
data_array = []
N_real_ev = br.GetEntries()
print(N_real_ev)
good_event_counter = 0
for ievent in range(N_real_ev):
    if good_event_counter>6000: break
    myevent.ClearEvent()
    br.GetEntry(ievent)
    event_container.SetEvent(myevent)
    if not myevent.GetNtrack(): continue
    do_continue = False
    for ipart in range(myevent.GetNtrack()):
        if myevent.GetEntry(ipart).GetMcId()==part_id and (myevent.GetEntry(ipart).GetEcore()/myevent.GetEntry(ipart).GetPtot()>0.6 or myevent.GetEntry(ipart).GetN0()>=0):
            do_continue = True
    if not do_continue:
        continue
    good_event_counter+=1
    event_container.correct_beam_offset()
    for ipart in range(myevent.GetNtrack()):
        if myevent.GetEntry(ipart).GetMcId()==part_id and (myevent.GetEntry(ipart).GetEcore()/myevent.GetEntry(ipart).GetPtot()>0.6 or myevent.GetEntry(ipart).GetN0()>=0):
            data_array.append([int(myevent.GetCentrality()),myevent.GetPreciseZ(),myevent.GetEntry(ipart).GetPtPrime(),\
                           myevent.GetEntry(ipart).GetEcore()/myevent.GetEntry(ipart).GetPtot(),myevent.GetEntry(ipart).GetN0(),\
                           myevent.GetEntry(ipart).GetDisp(),myevent.GetEntry(ipart).GetChi2(),myevent.GetEntry(ipart).GetNpe0(),\
                           myevent.GetEntry(ipart).GetProb(),myevent.GetEntry(ipart).GetPhi0Prime()   ])


data = pd.DataFrame(data_array, columns=('centrality','zvertex','pt', 'e/p', 'n0', 'disp', 'chi2', 'npe0', 'prob', 'phi0'))

99748438


In [9]:
data.head()

,centrality,zvertex,pt,e/p,n0,disp,chi2,npe0,prob,phi0
0,1,9.87306,0.656944,0.818837,-9999,-9999.0,-9999.0,-9999,0.200266,1.060966
1,83,-4.40653,0.592385,1.285034,-9999,-9999.0,-9999.0,-9999,0.313758,0.412905
2,16,5.78591,0.588732,0.617897,-9999,-9999.0,-9999.0,-9999,0.815753,2.971825
3,22,-6.40073,0.601085,0.905744,-9999,-9999.0,-9999.0,-9999,0.251313,1.209581
4,45,8.92402,0.415263,1.009133,-9999,-9999.0,-9999.0,-9999,0.933890,3.358011


In [10]:
print(data.size, data.shape, good_event_counter)

60140 (6014, 10) 6001


In [11]:
filepath = '/sphenix/user/mitrankov/'+filename
compression_options = dict(method='zip', archive_name=f'{filename}.csv')

In [12]:
data.to_csv(f'{filepath}.zip', compression=compression_options, index=False)

OSError: Cannot save file into a non-existent directory: '/sphenix/user/mitrankov'

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
import numpy as np
from sklearn.model_selection import GridSearchCV
parameters = {
    "loss":["log_loss"],
    "learning_rate": [0.01, 0.2],
    "min_samples_split": [0.01, 0.2],
    "max_depth":[3,8],
    "max_features":["log2","sqrt"],
    "criterion": ['squared_error'],
    "n_estimators":[2]
    }
clf3 = GridSearchCV(GradientBoostingClassifier(), parameters, cv=2, n_jobs=4)

X_train = np.array([[0, 0], [1, 1], [0, 1], [1, 0], [0, 1], [1, 0]])
y_train = np.array([0, 1, 0, 0, 0, 1])

# Fit the model
clf3.fit(X_train, y_train)

# Get decision function from clf3
clf_decision = clf3.decision_function(X_train)

# Initialize log odds predictions
log_odds_predictions = np.zeros(X_train.shape[0])

# Get the best estimator
best_estimator = clf3.best_estimator_

# Iterate over trees in the best estimator
for tree in best_estimator.estimators_:
    # Get leaf indices for training data
    leaf_indices = tree[0].apply(X_train)
    # Get values at each leaf (log odds)
    leaf_values = tree[0].tree_.value[:, 0, 0]  # Accessing the log odds correctly
    # Accumulate log odds for each instance
    log_odds_predictions += leaf_values[leaf_indices] * best_estimator.learning_rate

# Final log odds with the learning rate applied
mydecision = log_odds_predictions

# Output the differences
print("My Decision (Log Odds):", mydecision)
print("Decision Function Output:", clf_decision)

# Calculate the differences
difference = np.abs(mydecision - clf_decision)
print("Differences:", difference)
